# 日経225全銘柄の投資効率を検証
https://qiita.com/MandT500/items/f4aaafa0de824dab92c2
# pandas_datareader -> yfinance

In [1]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import time,sys,os
import joblib
today = datetime.date.today()

prod = '5y'
dys = 365
dates = '2021-06-27' # 用いるデータの入ったフォルダ

last = today +datetime.timedelta(days=-dys)
last = pd.to_datetime(last)

os.makedirs('./csv/'+str(today),exist_ok = True)

import requests

def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'NFuqiryl8uSLNdzCG4B0S0z9qTql7u8LCBP0aQCiJ90'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

In [2]:
"""TOPIX リストの読み込み"""
Core30 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_Core30_20210608.csv')
T100 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_100_20210618.csv')
T400 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_400_20210619.csv')
#S500 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S500_20210618.csv')
S1200 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S1200_20210620.csv')

S500 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S500_with_info_20210620.csv'\
                       ,).drop('日付',axis=1)

data =pd.DataFrame(S500).copy() 
#data = data.iloc[5:15]

data['code'] = [str(s)+".T" for s in data.code]
stocks = data['code']

#stocks = [str(s)+".T" for s in data.code]
#stocks.append("^N225")
tickers = yf.Tickers(" ".join(stocks))
stocks

0      1377.T
1      1379.T
2      1417.T
3      1662.T
4      1719.T
        ...  
488    9945.T
489    9946.T
490    9948.T
491    9956.T
492    9997.T
Name: code, Length: 493, dtype: object

In [3]:
closes= []

for i in range(len(tickers.tickers)):
    closes.append(tickers.tickers[i].history(period = prod).Close)

closes = pd.DataFrame(closes).T.ffill()
closes.columns = stocks
print(closes)


code             1377.T       1379.T       1417.T       1662.T      1719.T  \
Date                                                                         
2016-06-28  2157.744141  1656.983276   868.913391  1815.823975  466.802216   
2016-06-29  2176.005615  1639.731812   885.674194  1867.441406  479.556396   
2016-06-30  2130.832520  1624.205811   885.674194  1889.562988  481.256866   
2016-07-01  2212.528564  1650.082642   895.377808  1873.893433  480.406647   
2016-07-04  2153.899414  1656.983276   889.202820  1891.406494  476.155212   
...                 ...          ...          ...          ...         ...   
2021-06-22  3755.000000  1918.000000  1941.000000  1995.000000  827.000000   
2021-06-23  3755.000000  1914.000000  1913.000000  1995.000000  811.000000   
2021-06-24  3755.000000  1924.000000  1910.000000  2042.000000  812.000000   
2021-06-25  3755.000000  1931.000000  1913.000000  2067.000000  806.000000   
2021-06-28  3720.000000  1934.000000  1950.000000  2080.000000  

In [4]:
# 収益率を計算
df_returns = (closes - closes.shift(1) )/ closes.shift(1)
# 標準偏差，　分散，シャープ・レシオを計算
df_std = pd.DataFrame(df_returns.var(),columns=['Variance'])
df_mean = pd.DataFrame(df_returns.mean(),columns=['Mean'])
df_sharpR = pd.DataFrame(df_returns.mean() / df_returns.var(),
                         columns = ['Sharpe Ratio'])
# 結果を結合
df_result = pd.concat([df_std, df_mean, df_sharpR], axis=1)

# 標準偏差と収益率のグラフを作成
#df_result.plot.scatter(x='Variance',y='Mean')

# to csv
closes.to_csv('./csv/'+str(dates)+'/ttl_closes_'\
                    +str(len(stocks))+'_'+str(prod)+'.csv')
df_result.to_csv('./csv/'+str(today)+'/df_result_'\
                    +str(len(stocks))+'_ShapeR.csv')

In [5]:
stock_list = pd.DataFrame(data.set_index('code').iloc[:,0].str[:6],columns =['name'])

In [6]:
_df_result = pd.concat([stock_list,df_result],axis=1)
__df_result = _df_result[_df_result['Variance'] <1]

In [7]:
__df_result = __df_result.sort_values('Sharpe Ratio',ascending=False).iloc[:100]
__df_result

,name,Variance,Mean,Sharpe Ratio
code,,,,
8803.T,平和不動産,0.000327,0.001173,3.591613
7821.T,前田工繊,0.000370,0.001297,3.502540
5857.T,アサヒホール,0.000355,0.001239,3.492757
8511.T,日本証券金融,0.000260,0.000901,3.459511
9086.T,日立物流,0.000303,0.001030,3.403371
...,...,...,...,...
8892.T,日本エスコン,0.000580,0.001186,2.045676
4722.T,フューチャー,0.000562,0.001148,2.044875
6750.T,エレコム,0.000397,0.000812,2.043424


In [8]:
_df_result.sort_values('Sharpe Ratio',ascending=False)

,name,Variance,Mean,Sharpe Ratio
code,,,,
8803.T,平和不動産,0.000327,0.001173,3.591613
7821.T,前田工繊,0.000370,0.001297,3.502540
5857.T,アサヒホール,0.000355,0.001239,3.492757
8511.T,日本証券金融,0.000260,0.000901,3.459511
9086.T,日立物流,0.000303,0.001030,3.403371
...,...,...,...,...
8714.T,池田泉州ホー,0.000296,-0.000373,-1.262499
2193.T,クックパッド,0.000622,-0.000919,-1.476902
8358.T,スルガ銀行,0.000723,-0.001092,-1.510952


In [9]:
__df_result = __df_result.sort_values('Sharpe Ratio',ascending=False).iloc[:100]
__df_result


,name,Variance,Mean,Sharpe Ratio
code,,,,
8803.T,平和不動産,0.000327,0.001173,3.591613
7821.T,前田工繊,0.000370,0.001297,3.502540
5857.T,アサヒホール,0.000355,0.001239,3.492757
8511.T,日本証券金融,0.000260,0.000901,3.459511
9086.T,日立物流,0.000303,0.001030,3.403371
...,...,...,...,...
8892.T,日本エスコン,0.000580,0.001186,2.045676
4722.T,フューチャー,0.000562,0.001148,2.044875
6750.T,エレコム,0.000397,0.000812,2.043424
